# Задание по лекциям 1.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов**.

**(1 балл) Задание 1.** Реализовать функцию, принимающую на вход два набора 
$$
u = (u_1, \ldots, u_n),\ v = (v_1,\ldots,v_n)
$$
координат векторов (в стандартном базисе) из пространства $\mathbb{R}^n$, и выдающую 
- матрицу перехода от базиса $u$ к базису $v$, если оба этих набора являются базисами;
- None, иначе (альтернативный вариант — кидать исключение).

In [26]:
import numpy as np
import scipy.linalg as sla
from numpy import linalg as la
import math
from math import fabs

In [23]:
def trans_matrix(base_u,base_v):
  if sla.det(base_u)==0 or sla.det(base_v)==0:
    return None
  else:
    A = base_v@sla.inv(base_u)
    return A

base_u = np.array([[1,2,-1],[-1,-4,2],[5,1,0]]).reshape((3,3)).T
base_v = np.array([[1,0,0],[0,1,0],[0,0,1]]).reshape((3,3))
b= trans_matrix(base_u,base_v)
sla.inv(b)==trans_matrix(base_v,base_u)# зная свойство матрицы перехода, что матрица, обратная матрице перехода, равно матрице перехода в другую 
#сторону, проверим работу функции
base_u1 = np.array([[1,2,-1],[1,2,-1],[5,1,0]]).reshape((3,3)).T
trans_matrix(base_u1,base_v)# тут она ничего не вернула потому что base_u1 линейно зависим(не базис)

**(1 балл) Задание 2.** Реализовать функцию, принимающую на вход матрицу $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ (в стандартном базисе) и набор координат $u = (u_1,\ldots,u_n)$ вектор-столбцов, и выдающую 
- матрицу оператора $\varphi$ в базисе $u$, если $u$ является базисом;
- None, иначе (альтернативный вариант — кидать исключение).

In [22]:
def operator(A,base_u):
  if sla.det(base_u)==0:
    return None
  else:
    phi = sla.inv(base_u)@A@base_u#по теореме о переходе матрицы линейного преобразования из одного базиса в другой, с учетом, что базис стандартный
    return phi

base_u = np.array([[2,3,1],[3,4,1],[1,2,2]]).reshape((3,3)).T
A = np.array([[15,-11,5],[20,-15,8],[8,-7,6]]).reshape((3,3))
operator(A,base_u)# хромает точность, но я посмотрела, что машинный эпислон тут примерно покрывает косяки, поэтому здесь точность теряется
#уже на 14-15 знаке после запятой, поэтому все, что в степени меньше -13 это 0.


2.842170943040401e-14


array([[1.00000000e+00, 2.84217094e-14, 1.42108547e-14],
       [4.44089210e-16, 2.00000000e+00, 2.66453526e-15],
       [1.33226763e-15, 1.77635684e-15, 3.00000000e+00]])

**(1.5 балла) Задание 3.** Реализовать функции, которые по данной матрице $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ в стандартном базисе, выдадут:

- базис образа $\varphi$ (0.5 балла)
- базис ядра $\varphi$ (0.75 балла)
- размерности ядра и образа $\varphi$ (0.25 балла)

In [35]:
def core_n_im_n_dim(A):
  b=np.zeros((len(A)))
  rank_im=la.matrix_rank(A)
  rank_core=len(A)-rank_im
  if rank_core == 0:
    im_solv= sla.solve(A,b)
    return im_solv,0,rank_im,0
  else:
    my_num=la.eigvals(A)
    num=0
    for i in range(0,len(my_num)):
      if fabs(my_num[i])>1.e-15:
        num+=1
    U, S, V= sla.svd(A)
    im_solv= U[:,:num].T
    core_solv=V[num:,:]
    return im_solv, core_solv,rank_im,rank_core
# ну в общем на лекциях не был рассказан алгоритм по поиску этих базисов, поэтому я нашла в интернете то,
# почему представленнный выше алгоритм верный, сама я в этом разобралась ниже ссылка на что я опиралась
# https://math.stackexchange.com/questions/1771013/how-is-the-null-space-related-to-singular-value-decomposition

A=np.array([[1,2,1],[2,4,2],[1,3,0]])

im_solv, core_solv,rank_im,rank_core=core_n_im_n_dim(A)
print("Базис образа: ", im_solv,"\nБазис ядра: ", core_solv,"\nРазмерность образа = ",rank_im,"\nРазмерность ядра = ",rank_core)

Базис образа:  [[-0.39222212 -0.78444424 -0.4804259 ]
 [-0.214853   -0.42970599  0.87703532]] 
Базис ядра:  [[ 0.90453403 -0.30151134 -0.30151134]] 
Размерность образа =  2 
Размерность ядра =  1


**(1 балл) Задание 4.** Реализовать функцию, принимающую на вход матрицу $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ (в стандартном базисе) и набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую 
- True, если линейная оболочка векторов $(u_1,\ldots, u_k)$ является $\varphi$-инвариантным подпространством.
- False, иначе.

In [ ]:
def true_inv(A,u):
  rank_u= la.matrix_rank(u)
  new_u=np.column_stack((u,u@A))
  rank_new_u= la.matrix_rank(new_u)
  if rank_u==rank_new_u:
    return True
  else:
    return False
    